In [ ]:
# import packages
import pandas as pd
from datetime import datetime

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# read the datasets
orginal_data_path = 'original_data/'

links = pd.read_csv(orginal_data_path + 'links.csv')
links['imdb_id'] = links['imdbId'].apply(lambda x: f"tt{int(x):07d}" if pd.notnull(x) else None)

movies_cleaned = pd.read_csv('movies_cleaned.csv')

ratings = pd.read_csv(orginal_data_path + 'ratings.csv')
ratings['imdb_id'] = ratings['movieId'].map(dict(zip(links['movieId'], links['imdb_id'])))

ratings_small = pd.read_csv(orginal_data_path + 'ratings_small.csv')
ratings_small['imdb_id'] = ratings_small['movieId'].map(dict(zip(links['movieId'], links['imdb_id'])))

# keywords = pd.read_csv(orginal_data_path + 'keywords.csv')
# credits = pd.read_csv(orginal_data_path + 'credits.csv')

ratings['date_time'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [ ]:
genre_columns = [col for col in movies_cleaned.columns if col.startswith('genre')]
genre_columns = genre_columns[2:]

In [ ]:
test_user_id = 42

# Top Weighted Rating Recommendation

In [ ]:
# weighted_rating = (v*R+m*C)/(v+m)
# R = vote_average for a movie
# v = vote_count for a movie
# m = the threshold of vote_count for qualified movies
# C = average vote_average for all movies

In [ ]:
print('All movie count:', movies_cleaned.shape[0])
threshold = 0.75
qualify_movies = movies_cleaned[(movies_cleaned['vote_count'] > movies_cleaned['vote_count'].quantile(threshold)) & \
                                (movies_cleaned['vote_average'].notna())].copy()
print('Threshold for voting_count:', movies_cleaned['vote_count'].quantile(threshold))
print('Qualified movie count:', qualify_movies.shape[0])

In [ ]:
m = movies_cleaned['vote_count'].quantile(threshold)
C = movies_cleaned['vote_average'].mean()

def weighted_rating(x, m, C):
    v = x['vote_count']
    R = x['vote_average']
    return (v*R+m*C)/(v+m)

qualify_movies['weighted_rating'] = qualify_movies.apply(lambda x: weighted_rating(x, m, C), axis=1)

In [ ]:
# top 20 rating movies
used_columns = ['imdb_id', 'original_title', 'vote_count', 'vote_average']

top_20_movies = qualify_movies.sort_values('weighted_rating', ascending = False).head(20)[used_columns + ['weighted_rating']].copy().reset_index(drop = True)
top_20_movies

In [ ]:
# top rating movies by genre
top_10_genre_movies = pd.DataFrame()

for col in genre_columns:

    mv_df = qualify_movies[qualify_movies[col] == 1].sort_values('weighted_rating', ascending = False).head(10)[used_columns + ['weighted_rating']]
    mv_df['genre'] = col.split('_')[1]

    top_10_genre_movies = pd.concat([top_10_genre_movies, mv_df], ignore_index = True)

top_10_genre_movies

# Recommendation by Favorite Genre

In [ ]:
user_rating = ratings[ratings['userId'] == test_user_id].copy()
user_rating = user_rating.sort_values('rating', ascending = False)
user_rating = pd.merge(user_rating, movies_cleaned[['imdb_id', 'original_title'] + genre_columns], on = 'imdb_id', how='left')
user_rating

In [ ]:
# find the genres of the favorite movies
fav_genre = []
for genre in genre_columns:
    if user_rating[user_rating['rating'] == user_rating['rating'].max()][genre].sum() > 0:
        fav_genre.append(genre)

fav_genre

In [ ]:
# recommendations
qualify_movies[(qualify_movies[fav_genre].any(axis=1)) & (~qualify_movies['imdb_id'].isin(user_rating['imdb_id']))].\
            sort_values('weighted_rating', ascending = False).head(10)[used_columns + ['weighted_rating']]

In [ ]:
high_user_rating = user_rating[user_rating['rating'] >=  user_rating['rating'].quantile(0.75)]

In [ ]:
genre_sums = {}

for col in genre_columns:
    genre_sums[col] = high_user_rating[col].sum()

top_n = 3
sorted_vals = pd.Series(genre_sums).sort_values(ascending=False)

cutoff = sorted_vals.iloc[top_n - 1] 

top3_genres = sorted_vals[sorted_vals >= cutoff]

top3_genres

In [ ]:
test = pd.DataFrame(top3_genres, columns = ['origin'])
test['weighted_avg'] = top3_genres / top3_genres.unique().sum()
test['origin_count'] = test['origin'].map(test['origin'].value_counts())
test['weighted_avg_split'] = test['weighted_avg'] / test['origin_count']
test['movie_counts'] = test['weighted_avg_split'] * 10

In [ ]:
test

# Collaborative Filtering

In [ ]:
# create the user similarity matrix and get the top 10 similar users

rating_matrix = ratings_small.pivot_table(index='userId', columns='imdb_id', values='rating')

user_sim_matrix = pd.DataFrame(cosine_similarity(rating_matrix.fillna(0)), 
                               index = rating_matrix.index, 
                               columns = rating_matrix.index)

sim_users = user_sim_matrix[test_user_id].sort_values(ascending = False)[1:11].index

In [ ]:
# calculate the weighed average of the movie rating
weighted_sum = rating_matrix[rating_matrix.index.isin(sim_users)].\
               mul(user_sim_matrix.loc[user_sim_matrix.index.isin(sim_users), test_user_id], axis=0)

total_sim = rating_matrix[rating_matrix.index.isin(sim_users)].copy()
total_sim[total_sim.notna()] = 1
total_sim = total_sim.mul(user_sim_matrix.loc[user_sim_matrix.index.isin(sim_users), test_user_id], axis=0)

weighted_score = (weighted_sum.sum() / total_sim.sum()).round(4)

# combine weighted score with average similarity
score_sim = pd.DataFrame({'weighted_score': weighted_score, 'avg_sim': total_sim.mean()})

In [ ]:
# remove watched movies
test_id_watched = ratings_small[ratings_small['userId'] == test_user_id]['imdb_id']
score_sim = score_sim[~score_sim.index.isin(test_id_watched)].sort_values(['weighted_score', 'avg_sim'], ascending = [False, False])

In [ ]:
# get the top 10 score movies
cf_recommend_10 = score_sim.head(10).index


cf_recommend_10_df = movies_cleaned[movies_cleaned['imdb_id'].isin(cf_recommend_10)][used_columns].copy().merge(
                        score_sim[~score_sim.index.isin(test_id_watched)],
                        how='left',
                        left_on='imdb_id',
                        right_index=True).\
                        sort_values(['weighted_score', 'avg_sim', 'vote_average'], ascending = [False, False, False]).reset_index(drop = True)

cf_recommend_10_df